In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from hcmus.core import appconfig
from hcmus.lbs import LabelStudioConnector

2025-06-26 11:42:56.496 | INFO     | hcmus.core.appconfig:<module>:7 - Load DotEnv: True
/Volumes/Cucumber/Projects/item-identification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [10]:
lsb_connector = LabelStudioConnector(
    url=appconfig.LABEL_STUDIO_URL,
    api_key=appconfig.LABEL_STUDIO_API_KEY,
    project_id=appconfig.LABEL_STUDIO_PROJECT_MAPPING["val"],
    temp_dir=appconfig.LABEL_STUDIO_TEMP_DIR
)

In [11]:
tasks = lsb_connector.get_tasks()
dataset = lsb_connector.download_dataset(tasks)

2025-06-26 11:45:16.989 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:148 - Page size is too large, only 331 tasks available.
2025-06-26 11:45:16.989 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:152 - New `page_to` applied: 4
Loading tasks: 100%|██████████| 4/4 [00:01<00:00,  3.22it/s]
2025-06-26 11:45:18.236 | INFO     | hcmus.lbs._label_studio_connector:download_dataset:74 - No labels input, auto extract 82 labels.


In [12]:
label_dict = lsb_connector.extract_labels(tasks)

In [13]:
idx_to_label = list(label_dict.keys())
all_labels = []
for item in dataset:
    for label in item.get("target").get("labels"):
        all_labels.append(idx_to_label[label])

In [14]:
from collections import Counter
counts = Counter(all_labels)
len(counts)

82

In [15]:
import pandas as pd
from IPython.display import display, HTML

In [16]:
data = [(k, v) for k, v in counts.items()]
df = pd.DataFrame(data, columns=["Label", "Num Samples"])
df = df.sort_values("Num Samples")
display(HTML(df.to_html(index=False)))

Label,Num Samples
8934822801335-bia-tiger-crystal-4-6-phan-tram-lon-330ml,1
8935136865709-sua-tam-purite-duong-da-mem-min-huong-hoa-hong-rose-chai-850ml,1
8934868166825-dau-goi-sunsilk-natural-duong-ngan-gay-rung-650g,2
4902430805322-nuoc-rua-tay-diet-khuan-safeguard-trang-tinh-khiet-450ml,2
8934868166924-dau-goi-sunsilk-mem-muot-dieu-ky-chai-650g,3
8934868166870-dau-goi-sunsilk-ong-muot-rang-ngoi-chai-650g,3
8936048470036-mi-3-mien-mi-chay-la-da-65g-(goi),3
8935137601122-tra-xanh-thai-nguyen-100g,3
8935006538634-dau-tay-trang-hada-labo-sach-sau-duong-am-toi-uu-200ml,3
8934868170990-nuoc-rua-tay-lifebuoy-vitamin-sua-duong-am-tui-1kg,4
